In [ ]:
import os
from dotenv import load_dotenv

from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA

from langchain.globals import set_debug

# Variables

In [419]:
load_dotenv(override=True)

True

In [ ]:
file = os.getenv("FILE")
api_key = os.getenv("OPENAI_API_KEY")
mongo_url = os.getenv("MONGO_URL")
mongo_db = os.getenv("MONGO_DB")
mongo_collection = os.getenv("MONGO_COLLECTION")

# PDFLoader

In [421]:
loader = PyPDFLoader(file)
files = loader.load()

In [422]:
id = "45657525-1a48-4c82-88aa-a9f9037b2d4b"

In [423]:
for file in files:
    file.metadata['id'] = str(id)

# Recursive TextSplit

In [424]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

In [425]:
documents = recur_split.split_documents(files)

# Embbiding (OpenAI)

In [426]:
embeddings = OpenAIEmbeddings()

# Vector Store

In [428]:
client = MongoClient(mongo_url)
db = client[mongo_db]
collection = db[mongo_collection]

In [429]:
vectordb = MongoDBAtlasVectorSearch.from_documents( 
    documents=documents, 
    embedding= embeddings, 
    collection=collection,
    index_name="vector_index"
)

_id or id key found in metadata. Please pop from each dict and input as separate list.Retrieving methods will include the same id as '_id' in metadata.


# Retriever

In [430]:
retriever = vectordb.as_retriever(
    search_type='similarity', 
    search_kwargs = {
        "k": 5,
        'fetch_k': 30,
        'pre_filter': {'id': str(id)}
    }
)

# LLM (OpenAI)

In [431]:
llm = ChatOpenAI(model="gpt-4o", api_key=api_key, temperature=0)

# Prompt

In [432]:
prompt = ChatPromptTemplate.from_template(
"""
Você é um assistente de extração de dados educacionais.  
Sua tarefa é localizar **exatamente** os Códigos e Descrições **que já estão presentes no contexto abaixo** de acordo com os dados fornecidos.  
**Você NÃO deve criar, reescrever, resumir ou modificar nenhuma informação.**

{context}

REGRAS IMPORTANTES:
- Você **NÃO** pode inventar nenhum conteúdo que não esteja literalmente no contexto acima.
- Você deve retornar **apenas os códigos e descrições exatos e completos** que aparecem no contexto.
- **Não corte partes das descrições.**
- **Não relacione o conteúdo com códigos errados.**

FORMATO OBRIGATÓRIO DE SAÍDA (copiar literalmente do contexto):
Código: <código exato>  
Descrição: <descrição completa>

---

Pergunta do usuário:  
{question}
"""
)

# Chain

In [ ]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)

# Question

In [436]:
question = "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Porcentagem> e o **Objetivo da Aula** é <Uso para calcular juros>."

# Answer

In [437]:
chat_chain.invoke({"query": question})

{'query': '**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Porcentagem> e o **Objetivo da Aula** é <Uso para calcular juros>.',
 'result': 'Código: EF09MA05  \nDescrição: Resolver e elaborar problemas que envolvam porcentagens, com a ideia de aplicação de percentuais sucessivos e a determinação das taxas percentuais, preferencialmente com o uso de tecnologias digitais, no contexto da educação financeira.',
 'source_documents': [Document(id='689e88fa15c65a06b4a963e0', metadata={'_id': '689e88fa15c65a06b4a963e0', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250429004401', 'source': '../pdf/matematica.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'id': '45657525-1a48-4c82-88aa-a9f9037b2d4b'}, page_content='tecnologias digitais, no contexto da educação financeira.\nCódigo: EF09MA06\nDescrição: Compreender as funções como relações de dependência unívoca entre duas variáveis e suas\nrepresentações numéri